## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import modules as f # f is for function

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [27]:
f.predictions('joby')

JOBY 5m Interval Timestamp: 2024-11-05 16:38:57 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.237473,0.216701,0.254586,0.142857
prob_up,0.197567,0.255766,0.23439,0.125
prob_static,0.449888,0.385834,0.386797,0.625
prob_down,0.352545,0.3584,0.378813,0.25
precision,0.455338,0.440501,0.467562,0.387755
recall,0.404255,0.408124,0.404255,0.514507
f1,0.428279,0.423695,0.43361,0.442228
support,"[519.0, 517.0, 519.0]","[519.0, 517.0, 519.0]","[519.0, 517.0, 519.0]","[519.0, 517.0, 519.0]"


JOBY 15m Interval Timestamp: 2024-11-05 16:39:10 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,down,static,static
kelly_1:2.5,0.126882,0.121569,0.183333,0.140773
prob_up,0.155962,0.199247,0.195247,0.125
prob_static,0.513229,0.367481,0.423933,0.5
prob_down,0.330809,0.433272,0.38082,0.375
precision,0.376344,0.372549,0.416667,0.386266
recall,0.402299,0.32948,0.373563,0.517241
f1,0.388889,0.349693,0.393939,0.44226
support,"[171.0, 174.0, 173.0]","[171.0, 174.0, 173.0]","[171.0, 174.0, 173.0]","[171.0, 174.0, 173.0]"


JOBY 30m Interval Timestamp: 2024-11-05 16:39:14 


...

JOBY 1d Interval Timestamp: 2024-11-05 16:39:34 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,static,static,static
kelly_1:2.5,0.19322,0.165385,0.15082,0.066667
prob_up,0.252723,0.296937,0.237788,0.25
prob_static,0.315199,0.389339,0.425667,0.375
prob_down,0.432077,0.313724,0.336545,0.375
precision,0.423729,0.403846,0.393443,0.333333
recall,0.378788,0.333333,0.380952,0.428571
f1,0.4,0.365217,0.387097,0.375
support,"[65.0, 63.0, 66.0]","[65.0, 63.0, 66.0]","[65.0, 63.0, 66.0]","[65.0, 63.0, 66.0]"


JOBY 1wk Interval Timestamp: 2024-11-05 16:39:39 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,up
kelly_1:2.5,0.258824,0.258824,0.4,0.138462
prob_up,0.253161,0.209873,0.288819,0.5
prob_static,0.045545,0.332394,0.271522,0.0
prob_down,0.701294,0.457732,0.439659,0.5
precision,0.470588,0.470588,0.571429,0.384615
recall,0.615385,0.615385,0.615385,0.416667
f1,0.533333,0.533333,0.592593,0.4
support,"[12.0, 10.0, 13.0]","[12.0, 10.0, 13.0]","[12.0, 10.0, 13.0]","[12.0, 10.0, 13.0]"


ValueError: The test_size = 1 should be greater or equal to the number of classes = 3

###### Check Summary Output

In [ ]:
symbol='NVDA'
interval='1h'

f.transform(symbol, interval)

df = f.load_model_df(symbol,interval)

df.tail(7)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

# Select columns ending with 'z21'
z_columns = ['ac_z5', 'ac_z8', 'ac_z13', 'ac_z21', 'top_z21', 'body_z21', 'bottom_z21', 'vol_z21', 'pct_gap_up_down_z21', 'kma_sma40_diff_z21']
data_z21 = df[z_columns].dropna()  # Drop rows with missing values for simplicity

# Create an elbow plot to find the optimal number of clusters
inertia = []
cluster_range = range(1, 11)  # Test for clusters from 1 to 10

for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_z21)
    inertia.append(kmeans.inertia_)

# Plot the elbow curve
plt.figure(figsize=(10, 6))
plt.plot(cluster_range, inertia, marker='o')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.title('Elbow Plot for Optimal Number of Clusters')
plt.xticks(cluster_range)
plt.grid(True)
plt.show()


In [ ]:
optimal_k = 3  # Replace with the optimal number from the elbow plot
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
data_z21['cluster'] = kmeans.fit_predict(data_z21)

# Check the cluster distribution
print(data_z21['cluster'].value_counts())

In [ ]:
# Add the 'cluster' column back to the original DataFrame
df['cluster'] = data_z21['cluster']



In [ ]:
symbol='NVDA'
interval='1d'

# # Define Eastern Time Zone
# eastern = pytz.timezone('US/Eastern')

# # Get current time in Eastern Time Zone
# eastern_time = datetime.now(eastern)

# # Format the time to include hour, minute, and seconds
# time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

# print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
models

In [ ]:
models['XGBoost'].feature_importances_

In [ ]:
feature_names

In [ ]:
merged_list = sorted(zip(feature_names, models['XGBoost'].feature_importances_),
                     key=lambda x: x[1], 
                     reverse=True
                    )
merged_list

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')